# Test NWB Single Neurons File

Adapted from: https://pynwb.readthedocs.io/en/stable/tutorials/domain/ecephys.html

In [1]:
from datetime import datetime
from dateutil.tz import tzlocal

import numpy as np

from pynwb import NWBFile

from pynwb.ecephys import ElectricalSeries, SpikeEventSeries

In [2]:
np.random.seed(1234)

## Initialize NWB File

In [3]:
nwbfile = NWBFile('my first synthetic recording', 'EXAMPLE_ID', datetime.now(tzlocal()),
                  experimenter='Dr. Bilbo Baggins',
                  lab='Bag End Laboratory',
                  institution='University of Middle Earth at the Shire',
                  experiment_description='I went on an adventure with thirteen dwarves to reclaim vast treasures.',
                  session_id='LONELYMTN')

## Devices

In [4]:
# Create a device
device = nwbfile.create_device(name='electrode_recorder')

In [5]:
# Create an electrode group
electrode_name = 'tetrode1'
description = "an example tetrode"
location = "somewhere in the hippocampus"

electrode_group = nwbfile.create_electrode_group(electrode_name,
                                                 description=description,
                                                 location=location,
                                                 device=device)

In [6]:
# Add a collection of electrodes
for idx in [1, 2, 3, 4]:
    nwbfile.add_electrode(id=idx,
                          x=1.0, y=2.0, z=3.0,
                          imp=float(-idx),
                          location='CA1', filtering='none',
                          group=electrode_group)

## Electrode Table

In [7]:
# Grab a sub-selection of electrodes
electrode_table_region = nwbfile.create_electrode_table_region([0, 2], 'the first and third electrodes')

In [8]:
electrode_table_region

electrodes hdmf.common.table.DynamicTableRegion at 0x140222604738912
    Target table: electrodes hdmf.common.table.DynamicTable at 0x140222604739296

## Add time series data

In [9]:
# Set meta data
rate = 10.0

# Simulate some example data
data_len = 1000
ephys_data = np.random.rand(data_len * 2).reshape((data_len, 2))
ephys_timestamps = np.arange(data_len) / rate

In [10]:
# 
ephys_ts = ElectricalSeries('test_ephys_data',
                            ephys_data,
                            electrode_table_region,
                            timestamps=ephys_timestamps,
                            # Alternatively, could specify starting_time and rate as follows
                            # starting_time=ephys_timestamps[0],
                            # rate=rate,
                            resolution=0.001,
                            comments="This data was randomly generated with numpy, using 1234 as the seed",
                            description="Random numbers generated with numpy.random.rand")
nwbfile.add_acquisition(ephys_ts)

## Add units & spike events

In [11]:
# Add some units 
nwbfile.add_unit(id=1, electrodes=[0])
nwbfile.add_unit(id=2, electrodes=[0])

## Check File

In [12]:
# Check out the details of the example NWB file
nwbfile

root pynwb.file.NWBFile at 0x140222604603056
Fields:
  acquisition: {
    test_ephys_data <class 'pynwb.ecephys.ElectricalSeries'>
  }
  devices: {
    electrode_recorder <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    tetrode1 <class 'pynwb.ecephys.ElectrodeGroup'>
  }
  electrodes: electrodes <class 'hdmf.common.table.DynamicTable'>
  experiment_description: I went on an adventure with thirteen dwarves to reclaim vast treasures.
  experimenter: ['Dr. Bilbo Baggins']
  file_create_date: [datetime.datetime(2021, 3, 23, 10, 57, 58, 222646, tzinfo=tzlocal())]
  identifier: EXAMPLE_ID
  institution: University of Middle Earth at the Shire
  lab: Bag End Laboratory
  session_description: my first synthetic recording
  session_id: LONELYMTN
  session_start_time: 2021-03-23 10:57:58.222044-04:00
  timestamps_reference_time: 2021-03-23 10:57:58.222044-04:00
  units: units <class 'pynwb.misc.Units'>